In [ ]:
from pyomo.environ import *
from matplotlib import pyplot as plt
import numpy as np


from utils.model import init_model, getM1_rule
from utils.data import load_data, slope_bounds, intercept_bounds

In [ ]:
data = load_data('Data/DebrisFlow.txt')

CL, CU = slope_bounds(data)
DL, DU = intercept_bounds(data)

In [ ]:
b = 3
model = init_model(data)
model.b = Param(initialize=b)
model.B = RangeSet(1, b)
model.M = Param(model.I, initialize=getM1_rule(data))

model.c = Var(model.B, within=Reals, bounds=(CL, CU))
model.d = Var(model.B, within=Reals, bounds=(DL, DU))

model.e = Var(within=NonNegativeReals)
model.x = Var(model.I, model.B, within=Binary)

In [ ]:
@model.Constraint(model.I, model.B)
def error_constraint_leq(model, i, b):
    return model.Y[i] - model.c[b] * model.X[i] - model.d[b] <= model.e + model.M[i] * (1 - model.x[i, b])


@model.Constraint(model.I, model.B)
def error_constraint_geq(model, i, b):
    return model.Y[i] - model.c[b] * model.X[i] - model.d[b] >= -model.e - model.M[i] * (1 - model.x[i, b])


@model.Constraint(model.I)
def assignment_constraint(model, i):
    return sum(model.x[i, b] for b in model.B) == 1

model.obj = Objective(expr=model.e, sense=minimize)

In [ ]:
result = SolverFactory('glpk', tee=True, keepfiles=False).solve(model)

if result.solver.termination_condition != TerminationCondition.optimal:
    raise Exception('Solver did not find an optimal solution')

print(f'Objective: {model.obj():.2f}')

In [ ]:
data_clusters = np.array([sum(model.x[i, b].value * (b - 1) for b in model.B) for i in model.I])

In [ ]:
plt.scatter(data[:, 0], data[:, 1], c=data_clusters)
for b in model.B:
    plt.plot(data[:, 0], model.c[b].value * data[:, 0] + model.d[b].value, c='red', alpha=0.5)
plt.show()